In [0]:
dbutils.widgets.text("env","")

In [0]:
env = dbutils.widgets.get('env')
checkpointLocation = "/Volumes/workspace/storage/containers/checkpoints/"

In [0]:
def read_bonze_table(env):
    print(f"Reading bronze table taffic_analysis_{env}.bronze.raw_traffic")
    df_bronze_traffic = (spark.readStream
                    .table(f"traffic_analysis_{env}.bronze.raw_traffic"))
    print("Reading success")
    return df_bronze_traffic

##Handling Duplicates

In [0]:
def handle_duplicates(df):
    de_dup_df = df.dropDuplicates()
    print("Reoving Duplicates")
    print("*****************************************************************")
    return de_dup_df

## Handling Nulls

In [0]:
def handle_nulls(df, all_columns):
    print("Rplacing null values in the string columns witth Unknown")
    df_string = df.na.fill("Unknown", all_columns)
    print("Rplacing null values in the numerical columns witth 0")
    df_null = df.na.fill(0, all_columns)
    print("Nulls in string and numerical columns are handled successfully")
    return df_null

## Getting Count of Electrical Vehicles

In [0]:
from pyspark.sql.functions import col
def count_of_ev(df):
    print("Creating Electric_Vehicle_Count column", end = "")
    df_ev= df.withColumn("Electric_Vehicle_Count", col("EV_Car") + col("EV_Bike"))
    print("Success")
    return df_ev


##Getting count of Motor Vehicles

In [0]:
def count_of_mv(df):
    print("Creating Motor_Vehicle_Count column", end = "")
    df_mv= df.withColumn("Motor_Vehicle_Count", col("Pedal_cycles") + col("Two_wheeled_motor_vehicles") + col("Cars_and_taxis") + col("Buses_and_coaches") + col("LGV_Type") + col("HGV_Type") + col("Electric_Vehicle_Count"))
    print("Success")
    return df_mv

In [0]:
from pyspark.sql.functions import current_timestamp
def create_transformed_time(df):
    print("Creating TransformedTimestamp column", end = "")
    df_ct= df.withColumn("Transformed_Time", current_timestamp()
                         )
    print("Success")
    return df_ct

In [0]:
def writeTrafficSilverTable(streaming_df,env):
    print("Writing to silver table")

    writeStream = (streaming_df.writeStream
                .format("delta")
                .option("checkpointLocation", f"{checkpointLocation}/Traffic_Silver/checkpoint")
                .outputMode("append")
                .queryName("SilverTrafficWriteStream")
                .trigger(availableNow = True)
                .toTable(f"traffic_analysis_{env}.silver.traffic"))
    writeStream.awaitTermination()
    print(f"Writing traffic_analysis_{env}.silver.silver_traffic Success")


## Calling All Functions

In [0]:
df_bronze_traffic = read_bonze_table(env)
deduped_df = handle_duplicates(df_bronze_traffic)

all_columns = deduped_df.schema.names
df_null = handle_nulls(deduped_df, all_columns)

df_ev = count_of_ev(df_null)
df_mv = count_of_mv(df_ev)

df_ct = create_transformed_time(df_mv)
writeTrafficSilverTable(df_ct,env)
